In [2]:
import tushare as ts
import pandas as pd
import numpy as np
import os
import time
import tqdm
import talib as ta
import mplfinance as mpf
import matplotlib.pyplot as plt
from datetime import datetime
from pathlib import Path
import shutil


In [3]:
stock_basic_df = pd.read_csv("./data/stock_basic_df.csv")
ggt_df = pd.read_csv("./data/ggt_basic_df.csv")
etf_df = pd.read_csv("./data/etf_basic_df.csv")
fetf_df = pd.read_csv("./data/fetf_basic_df.csv")
basic_df = pd.read_csv("./data/basic_df.csv")

def chnname(code):
    name = ''
    if code in basic_df['ts_code'].values:
        name = basic_df.loc[basic_df['ts_code'] == code, 'name'].values[0]
    elif code in fetf_df['ts_code'].values:
        name = fetf_df.loc[fetf_df['ts_code'] == code, 'name'].values[0]
    return name
def chnindustry(code):
    industry = ''
    if code in basic_df['ts_code'].values:
        industry = basic_df.loc[basic_df['ts_code'] == code, 'industry'].values[0]
    return industry


In [4]:
industry_df = pd.read_csv("./data/industry.csv")
buy_list = pd.read_csv("./data/buy_list.csv")['Column1'].tolist()
up_list = pd.read_csv("./data/up_list.csv")['Column1'].tolist()
daily_data_dir = './data/daily'


In [5]:
def apply_rsi(df, period=24):
    df = df.sort_values(by="trade_date", ascending=True)
    df["rsi12"] = ta.RSI(df["close"].values, timeperiod=12)
    df["rsi60"] = ta.RSI(df["close"].values, timeperiod=60)
    df["rsi120"] = ta.RSI(df["close"].values, timeperiod=120)
    df['min_rsi'] = df[['rsi12', 'rsi60', 'rsi120']].min(axis=1)

    return df

In [6]:
# 计算每个组的比例
close_greater_than_ma60 = industry_df['close'] > industry_df['ma60']
close_greater_than_ma20 = industry_df['close'] > industry_df['ma20']
close_greater_than_ma5 = industry_df['close'] > industry_df['ma5']

industry_proportion_ma60 = close_greater_than_ma60.groupby(industry_df['industry']).mean() * 100
industry_proportion_ma20 = close_greater_than_ma20.groupby(industry_df['industry']).mean() * 100
industry_proportion_ma5 = close_greater_than_ma5.groupby(industry_df['industry']).mean() * 100

# 创建包含比例的 DataFrame
industry_proportion = pd.DataFrame({
    'MA5 Proportion': round(industry_proportion_ma5, 2),
    'MA20 Proportion': round(industry_proportion_ma20, 2),
    'MA60 Proportion': round(industry_proportion_ma60, 2),
})
industry_proportion.sort_values(by='MA20 Proportion', ascending=False, inplace=True)

# 计算每个组的行数
industry_counts = close_greater_than_ma5.groupby(industry_df['industry']).size().reset_index(name='MA5 Count')

# 将比例和行数合并到一个 DataFrame 中
industry_proportion_with_counts = pd.merge(industry_proportion, industry_counts, on='industry')

# 设置显示选项以显示所有行
pd.set_option('display.max_rows', None)

# 输出结果
industry_proportion_with_counts

,industry,MA5 Proportion,MA20 Proportion,MA60 Proportion,MA5 Count
0,渔业,66.67,100.00,100.00,6
1,铅锌,100.00,100.00,100.00,13
2,出版业,96.55,100.00,96.55,29
3,啤酒,87.50,100.00,100.00,8
4,超市连锁,100.00,100.00,100.00,9
5,纺织机械,90.91,100.00,100.00,11
6,铁路,57.14,100.00,100.00,7
7,旅游景点,88.24,100.00,94.12,17
8,公路,25.00,100.00,100.00,4
9,旅游服务,50.00,100.00,100.00,8


In [7]:
industry_list = ['银行','普钢','煤炭开采','小金属','焦炭加工','火力发电']
industry_proportion_with_counts[industry_proportion_with_counts['industry'].isin(industry_list)]

,industry,MA5 Proportion,MA20 Proportion,MA60 Proportion,MA5 Count
12,普钢,69.23,100.00,100.00,26
15,焦炭加工,50.00,100.00,100.00,8
16,银行,92.86,97.62,100.00,42
52,火力发电,71.88,84.38,90.62,32
88,煤炭开采,24.00,72.00,64.00,25
109,小金属,58.18,45.45,89.09,55


In [8]:
# hold and recent sell
hold_dict = {
    '000028.SZ': 29.78, # 国药一致
    '000157.SZ': 7.00, # 中联重科
    '000537.SZ': 9.45, # 中绿电
    '001234.SZ': 16.25, # 泰慕士
    '001260.SZ': 11.88, # 坤泰
    '001299.SZ': 13.73, # 美能能源
    '001366.SZ': 11.64, # 播恩
    '001376.SZ': 15.8, # 百通
    '001979.SZ': 11.65, # 招商蛇口
    '002546.SZ': 4.23, # 新联电子
    '159509.SZ': 1.546, # 纳指科技
    '159529.SZ': 1.404, # 标普消费
    '159687.SZ': 1.292, # 亚太精选
    '159869.SZ': 1.064, # 游戏 etf
    '159941.SZ': 1.203, # 纳指
    '164824.SZ': 1.579,  # 印度
    '300784.SZ': 68.90, # 利安
    '300998.SZ': 17.20, # 宁波方正
    '301107.SZ': 28.84, # 瑜欣电子
    '301212.SZ': 26.35, # 联盛化学
    '501018.SH': 1.203, # 南方原油
    '501300.SH': 0.970, # 美元债
    '511090.SH': 118.687, # 30年国债
    '511520.SH': 112.668, # 政金债
    '513030.SH': 1.487, # 德国
    '513080.SH': 1.508, # 法国
    '513290.SH': 1.225, # 纳指生物
    '513400.SH': 1.197, # 道琼斯
    '513500.SH': 2.23, # 标普500
    '513520.SH': 1.460, # 日经
    '513730.SH': 1.412, # 东南亚
    '513800.SH': 1.410, # 东证
    '515790.SH': 0.894, # 光伏
    '516160.SH': 2.062, # 新能源
    '518880.SH': 5.941, # 黄金
    '562800.SH': 0.562, # 稀有金属
    '588290.SH': 1.501, # 芯片科创
    '600016.SH': 4.09, # 民生银行
    '600096.SH': 22.09, # 云天化
    '600325.SH': 6.61, # 华发股份
    '600373.SH': 13.51, # 中文传媒
    '600585.SH': 26.25, # 海螺水泥
    '600780.SH': 6.00, # 通宝能源
    '600900.SH': 28.39, # 长江电力
    '601068.SH': 4.75, # 中铝
    '601107.SH': 5.20, # 四川成渝
    '601117.SH': 8.38, # 中国化学
    '601666.SH': 10.13, # 平煤股份
    '603056.SH': 14.4, # 德邦股份
    '603167.SH': 8.71, # 渤海轮渡
    '603172.SH': 14.81, # 万丰
    '603588.SH': 5.4, # 高能环境
    '688036.SH': 96.98, # 传音控股
    '688443.SH': 30.56, # 智翔金泰
    '688530.SH': 21.67, # 欧莱
    '688638.SH': 30.07, # 誉辰智能
    '688695.SH': 34.91, # 中创
}
recent_sell_dict = {
    #12.10
    '300995.SZ': 19.47, # 奇德新材
    '600036.SH': 38.92, # 招商银行
    '600064.SH': 8.33, # 南京高科
    #12.9
    '000543.SZ': 8.36, # 皖能电力
    '002508.SZ': 23.01, # 老板电器
    '600328.SH': 8.49, # 中盐化工
    #12.6
    '603280.SH':23.71, # 南方路机
    '560010.SH':2.58, # 1000
    #12.5
    '601669.SH': 5.5, # 中国电建
    #12.4
    '600089.SH':13.05, # 特变电工
    '601077.SH':5.69, # 渝农商行
    '002394.SH':8.21, # 联发股份
    # 12.3
    '600894.SH': 13.36, # 广日股份
    '002233.SZ': 7.71, # 塔牌
    '002191.SZ': 4.67, # 劲嘉
    '000517.SZ': 2.55, # 荣安
    '000504.SZ': 10.39, # 南华
    # 12.2
    '600153.SH': 10.8, # 建发
    # 11.28
    '600162.SH': 1.9 , # 香江
    # 11.27
    '515220.SH':1.166, # 煤炭
    # 11.26
    '601006.SH':6.66, # 大秦铁路
    # 11.25
    '513090.SH':1.479, # 香港证券
    '603816.SH':28.06, # 顾家家居
    '600008.SH':3.25 , # 首创
    '301359.SH':23.35 , # 东南
    '159922.SZ':6.065 , # 500
    '512200.SH':1.569 , # 地产
    '512480.SH':0.961 , # 半导体
    # 11.21
    '001256.SZ': 18.28, # 炜冈
    '001259.SZ': 24.02, # 利仁
    '601069.SH': 12.64, # 西部黄金
    # 11.20
    '603272.SH': 13.93, # 联翔
    '301234.SZ': 27.7, # 五洲医疗
    '001209.SZ': 14.22, # 洪兴
    '301309.SZ': 27.85, # 万德凯
    '001226.SZ': 25.56, # 拓山
    '301061.SZ': 55.61, # 匠心家居
    '000581.SZ': 18.29, # 威孚高科
    '603048.SH': 14.96, # 浙江黎明
    '688750.SH': 29.85, # 金天
    '002895.SZ': 21.4, # 川恒股份
    # 11.19
    '301043.SZ': 27.8, # 绿岛风
    '301388.SZ': 22.58, # 欣灵电气
    '001368.SZ': 19.55, # 通达创智
    '603216.SH': 10.49, # 梦天家居
    '603307.SH': 31.25, # 金泉
    '601919.SH': 14.6, # 中远海控
}

def apply_info(df):
    df['state'] = ''
    df['per'] = 0
    for i in df.index:
        ts_code = df.loc[i, 'ts_code']
        if ts_code in hold_dict:
            df.loc[i, 'state'] = 'hold'
            daily_df = pd.read_csv(os.path.join(daily_data_dir, f"{ts_code}.csv"))
            daily_df = daily_df.sort_values(by="trade_date", ascending=True)
            close = daily_df.iloc[-1]['close']
            df.loc[i, 'per'] = (close - hold_dict[ts_code]) / hold_dict[ts_code] * 100
        elif ts_code in recent_sell_dict:
            df.loc[i, 'state'] = 'sell'
            daily_df = pd.read_csv(os.path.join(daily_data_dir, f"{ts_code}.csv"))
            daily_df = daily_df.sort_values(by="trade_date", ascending=True)
            close = daily_df.iloc[-1]['close']
            df.loc[i, 'per'] = (close - recent_sell_dict[ts_code]) / recent_sell_dict[ts_code] * 100
            


In [9]:
mini_df = industry_df[industry_df['pe'] > 0]
mini_df = mini_df[mini_df['ts_code'].isin(up_list)]    
print(len(mini_df))
base_mv_per = industry_df["circ_mv"].quantile(0.1)
mini_df = mini_df[mini_df["circ_mv"] < base_mv_per]
mini_df['circ_mv'] = mini_df['circ_mv'] / 10000
print(base_mv_per)
print(len(mini_df))


for i in mini_df.index:
    ts_code = mini_df.loc[i, 'ts_code']
    df = pd.read_csv(os.path.join(daily_data_dir, f"{ts_code}.csv"))
    df = apply_rsi(df)
    df = df.sort_values(by="trade_date", ascending=False)
    df = df.head(1)
    mini_df.loc[i, 'rsi12'] = df['rsi12'].iloc[0]
    mini_df.loc[i, 'rsi60'] = df['rsi60'].iloc[0]
    mini_df.loc[i, 'rsi120'] = df['rsi120'].iloc[0]
    mini_df.loc[i, 'min_rsi'] = df['min_rsi'].iloc[0]

mini_df["min_rsi"] = mini_df['rsi12'] + mini_df['rsi60'] + mini_df['rsi120']
mini_df = mini_df.sort_values(by="min_rsi", ascending=True)   
apply_info(mini_df) 

# mini_df = mini_df.sort_values(by='circ_mv')
# mini_df = mini_df.drop_duplicates(subset='industry', keep='first')
mini_df.to_csv("data/mini_df.csv")
mini_df[['ts_code', 'name','industry','act_ent_type','state','per','circ_mv','pe','pb','dv_ttm','rsi12','rsi60','rsi120','min_rsi']].head(30)
# mini_df[['ts_code', 'name','industry','circ_mv','pe','pb','dv_ttm']].head(20)


2745
164545.56513
302


,ts_code,name,industry,act_ent_type,state,per,circ_mv,pe,pb,dv_ttm,rsi12,rsi60,rsi120,min_rsi
2515,300784.SZ,利安科技,塑料,民营企业,hold,-1.640058,9.528462,50.3449,4.9955,0.2951,52.200367,43.633777,39.185680,135.019824
1746,301529.SZ,福赛科技,汽车配件,民营企业,,0.000000,14.247851,33.7141,2.2849,0.8971,42.694591,49.706832,47.274582,139.676005
2621,301392.SZ,汇成真空,专用机械,民营企业,,0.000000,15.914250,87.2397,9.6375,0.3535,48.518186,48.363841,44.992040,141.874067
2475,001239.SZ,永达股份,工程机械,民营企业,,0.000000,7.802589,42.0674,3.0237,0.0000,44.941440,49.562082,48.072288,142.575809
1886,688693.SH,锴威特,半导体,NaN,,0.000000,13.522077,147.4100,2.7194,0.0000,45.421848,51.318126,48.021505,144.761479
2035,301539.SZ,宏鑫科技,汽车配件,民营企业,,0.000000,7.725600,42.3651,4.0540,0.9579,54.639750,47.251281,43.008356,144.899386
889,001378.SZ,德冠新材,塑料,民营企业,,0.000000,16.106373,27.2036,1.7413,1.8174,56.780556,49.083849,40.027456,145.891861
3403,301516.SZ,中远通,电气设备,民营企业,,0.000000,13.382456,81.0930,3.8344,0.9649,48.052978,50.259101,48.381260,146.693339
748,301587.SZ,中瑞股份,电气设备,民营企业,,0.000000,8.219867,29.1587,1.9784,0.0000,51.257392,49.347263,46.274310,146.878965
4516,603231.SH,索宝蛋白,食品,民营企业,,0.000000,8.194454,21.8019,1.7766,2.9206,50.884080,49.577497,46.477820,146.939397


In [10]:
ex_dict = {
    '000517.SZ': '荣安地产,民',
    '000672.SZ': '上峰水泥,民',
    '002191.SZ': '劲嘉股份,民营',
    '002233.SZ': '塔牌集团,民营',
    '002478.SZ': '常宝股份,民营,钢加工',
    '002623.SZ': '亚玛顿,民营',
    '002727.SZ': '一心堂,民营',
    '600884.SH': '杉杉股份,民营',
    '601339.SH': '百隆东方,民',
    '601886.SH': '江河集团,民营',
    '603588.SH': '高能环境,民营',  
    '600998.SH': '九州通,民营',
    '002539.SZ': '云图控股,民营',
    '601877.SH': '正泰电器,民营',
    '601222.SH': '林洋能源,民营',
    '301276.SZ': '嘉曼服饰,民营',
    '603368.SH': '柳药股份,民营',
    '603035.SH': '常熟汽饰,民',
    '603599.SH': '广信股份,民营',
}

base_df_per = industry_df["pb"].quantile(0.1)
base_pb = 1 if base_df_per < 1 else base_df_per
print("base_pb: ", base_pb, "base_df_per: ",base_df_per)

pb_df = industry_df[industry_df["pb"] <= base_pb]
print(len(pb_df))
pb_df = pb_df[pb_df['pe'] > 0]
print(len(pb_df))
pb_df = pb_df[pb_df['ts_code'].isin(up_list)]
print(len(pb_df))
pb_df.loc[:, 'circ_mv'] = pb_df['circ_mv'] / 10000
pb_df = pb_df[~pb_df['ts_code'].isin(list(ex_dict.keys()))]
pb_df = pb_df[pb_df['dv_ttm'] > 2]
print(len(pb_df))

for i in pb_df.index:
    ts_code = pb_df.loc[i, 'ts_code']
    df = pd.read_csv(os.path.join(daily_data_dir, f"{ts_code}.csv"))
    df = apply_rsi(df)
    df = df.sort_values(by="trade_date", ascending=False)
    df = df.head(1)
    pb_df.loc[i, 'rsi12'] = df['rsi12'].iloc[0]
    pb_df.loc[i, 'rsi60'] = df['rsi60'].iloc[0]
    pb_df.loc[i, 'rsi120'] = df['rsi120'].iloc[0]
    pb_df.loc[i, 'min_rsi'] = df['min_rsi'].iloc[0]

pb_df["min_rsi"] = pb_df['rsi12'] + pb_df['rsi60'] + pb_df['rsi120']
pb_df = pb_df.sort_values(by="min_rsi", ascending=True)  
apply_info(pb_df)   
# pb_df = pb_df.drop_duplicates(subset='industry', keep='first')
pb_df.to_csv("data/pb_df.csv")

pb_df[['ts_code', 'name','industry','act_ent_type','state','per','circ_mv','pe','pb','dv_ttm','rsi12','rsi60','rsi120','min_rsi']].head(30)

base_pb:  1.1392 base_df_per:  1.1392
506
421
308
151


,ts_code,name,industry,act_ent_type,state,per,circ_mv,pe,pb,dv_ttm,rsi12,rsi60,rsi120,min_rsi
2290,600325.SH,华发股份,区域地产,地方国企,hold,0.605144,170.987501,9.9583,0.8730,5.5639,49.091092,51.292322,49.922736,150.306149
799,000028.SZ,国药一致,医药商业,NaN,hold,0.167898,142.603423,10.3813,0.9172,2.2796,50.683423,50.452776,50.017732,151.153931
4295,601107.SH,四川成渝,路桥,地方国企,hold,0.384615,112.895028,13.4485,0.9930,4.5977,48.153235,51.175172,51.840552,151.168960
482,600780.SH,通宝能源,火力发电,NaN,hold,1.666667,69.936654,10.2882,0.9254,2.9508,51.194092,51.020900,49.109865,151.324858
2571,601666.SH,平煤股份,煤炭开采,NaN,hold,1.875617,254.836764,6.3816,1.0401,9.4958,51.988318,50.082929,49.790895,151.862142
3346,001979.SZ,招商蛇口,全国地产,NaN,hold,-3.690987,933.788702,16.0873,1.0397,2.8520,47.704778,53.167730,52.061964,152.934472
4535,601117.SH,中国化学,建筑工程,NaN,hold,-2.028640,498.312581,9.2433,0.8293,2.1681,48.183774,53.119671,52.432701,153.736146
3034,601326.SH,秦港股份,港口,NaN,,0.000000,166.038809,12.7351,1.0130,2.3782,50.939148,52.202497,52.378892,155.520538
3566,601390.SH,中国中铁,建筑工程,NaN,,0.000000,1345.902602,4.8714,0.5498,3.1866,51.583097,52.428197,51.696630,155.707924
899,601618.SH,中国中冶,建筑工程,NaN,,0.000000,606.989052,8.1265,0.6903,2.1176,51.561269,52.636155,51.688454,155.885877


In [11]:
buy_df = pd.DataFrame()

for b in buy_list:
    # 从 daily 里读 csv,只读第一行然后按 circ_mv 排序
    df = pd.read_csv(os.path.join(daily_data_dir, f"{b}.csv"))
    df = apply_rsi(df)
    
    df = df.sort_values(by="trade_date", ascending=False)
    df = df.head(1)
    ts_code = df['ts_code'].iloc[0]
    name = chnname(ts_code)
    industry = chnindustry(ts_code)
    df['name'] = name
    df['industry'] = industry
    buy_df = pd.concat([buy_df, df])

buy_df["min_rsi"] = buy_df['rsi12'] + buy_df['rsi60'] + buy_df['rsi120']
buy_df = buy_df.sort_values(by="min_rsi", ascending=True)    
buy_df['circ_mv'] = buy_df['circ_mv'] / 1e4
buy_df.reset_index(inplace=True)
apply_info(buy_df)
buy_df.to_csv("data/buy_df.csv")
print(len(buy_df))
buy_df[['ts_code', 'name','industry','state','per','circ_mv','rsi12','rsi60','rsi120','min_rsi']].head(50)

55


,ts_code,name,industry,state,per,circ_mv,rsi12,rsi60,rsi120,min_rsi
0,600225.SH,*ST卓朗,软件服务,,0,47.041791,40.485610,43.198050,43.711082,127.394741
1,02255.HK,海昌海洋公园,NaN,,0,0.000000,56.344487,46.156452,46.316426,148.817365
2,01066.HK,威高股份,NaN,,0,0.000000,64.606341,51.387362,47.391291,163.384994
3,002690.SZ,美亚光电,专用机械,,0,69.501483,63.438011,52.158326,49.496070,165.092407
4,603787.SH,新日股份,摩托车,,0,26.466536,62.177602,53.585114,50.456421,166.219136
5,600872.SH,中炬高新,食品,,0,186.039392,63.397251,53.932893,50.224204,167.554348
6,688665.SH,四方光电,电器仪表,,0,33.973940,64.044777,53.765777,49.796712,167.607265
7,002810.SZ,山东赫达,化工原料,,0,47.488335,62.889719,54.098252,50.722527,167.710498
8,300855.SZ,图南股份,小金属,,0,78.219289,64.321093,53.260509,50.898410,168.480011
9,000887.SZ,中鼎股份,汽车配件,,0,178.228654,62.517337,53.924744,52.314130,168.756211


In [12]:
up_df = pd.DataFrame()

for b in tqdm.tqdm(up_list, desc="Processing"):
    # 从 daily 里读 csv,只读第一行然后按 circ_mv 排序
    df = pd.read_csv(os.path.join(daily_data_dir, f"{b}.csv"))
    df = apply_rsi(df)
    df = df.sort_values(by="trade_date", ascending=False)
    df = df.head(1)
    ts_code = df['ts_code'].iloc[0]
    name = chnname(ts_code)
    industry = chnindustry(ts_code)
    df['name'] = name
    df['industry'] = industry
    df['name'] = name
    up_df = pd.concat([up_df, df])
    
print(len(up_df))
up_df["min_rsi"] = up_df['rsi12'] + up_df['rsi60'] + up_df['rsi120']
up_df = up_df.sort_values(by="min_rsi", ascending=True)    
up_df['circ_mv'] = up_df['circ_mv'] / 1e4
up_df.reset_index(inplace=True)
apply_info(up_df)
up_df.to_csv("data/up_df.csv")
up_df[['ts_code', 'name', 'industry','state','per','circ_mv','rsi12','rsi60','rsi120','min_rsi']].head(50)

Processing: 100%|██████████| 3920/3920 [00:16<00:00, 239.08it/s]


3920


,ts_code,name,industry,state,per,circ_mv,rsi12,rsi60,rsi120,min_rsi
0,600225.SH,*ST卓朗,软件服务,,0.000000,47.041791,40.485610,43.198050,43.711082,127.394741
1,02407.HK,高视医疗,NaN,,0.000000,0.000000,46.137186,45.326309,41.944656,133.408151
2,00314.HK,思派健康,NaN,,0.000000,0.000000,41.122044,46.169289,46.709523,134.000857
3,03800.HK,协鑫科技,NaN,,0.000000,0.000000,35.062080,49.025003,50.178048,134.265131
4,300784.SZ,利安科技,塑料,hold,-1.640058,9.528462,52.200367,43.633777,39.185680,135.019824
5,00968.HK,信义光能,NaN,,0.000000,0.000000,42.822651,47.148881,48.246775,138.218307
6,002459.SZ,晶澳科技,电气设备,,0.000000,519.369459,38.051428,52.039078,49.092117,139.182623
7,601012.SH,隆基绿能,电气设备,,0.000000,1335.250227,39.770734,51.062042,48.534871,139.367648
8,01164.HK,中广核矿业,NaN,,0.000000,0.000000,42.753254,47.411210,49.253074,139.417539
9,301529.SZ,福赛科技,汽车配件,,0.000000,14.247851,42.694591,49.706832,47.274582,139.676005


In [14]:
f_df = pd.DataFrame()

for c in fetf_df['ts_code']:
    name = chnname(c)
    if c in up_list:
        df = pd.DataFrame({'ts_code': [c], 'name': [name],'aim': 'long'})
        f_df = pd.concat([f_df, df], ignore_index=True)
    else:
        df = pd.DataFrame({'ts_code': [c], 'name': [name],'aim': ''})
        f_df = pd.concat([f_df, df], ignore_index=True)
        if c in hold_dict:
            print("sell : " + c)

apply_info(f_df)
print(len(f_df),round(100*0.5/len(f_df),2))
money = 900000
f = money* 0.5
print(f,f/len(f_df))
f3 = money*0.5/3
print(f3)
f_df


34 1.47
450000.0 13235.29411764706
150000.0


,ts_code,name,aim,state,per
0,513090.SH,香港证券ETF,,sell,8.789723
1,513180.SH,恒生科技指数ETF,long,,0.000000
2,513050.SH,中概互联网ETF,,,0.000000
3,513310.SH,中韩半导体ETF,,,0.000000
4,159920.SZ,恒生ETF,long,,0.000000
5,513120.SH,港股创新药ETF,,,0.000000
6,513060.SH,恒生医疗ETF,,,0.000000
7,159941.SZ,纳指ETF,long,hold,-0.498753
8,159509.SZ,纳指科技,long,hold,-1.034929
9,159529.SZ,标普消费,long,hold,-1.282051
